## Dolly 3B Model - by Databricks
---
1. Model Name: Dolly-v2-3b
2. Model Parameters: 3 Billion
3. Training: Instruction-tuned Model
4. Link: https://huggingface.co/databricks/dolly-v2-3b
---

In [ ]:
# install dependencies
!pip install transformers
!pip install sentencepiece
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import torch                        # allows Tensor computation with strong GPU acceleration
from transformers import pipeline   # fast way to use pre-trained models for inference
import os

In [ ]:
dolly_pipeline = pipeline(model="databricks/dolly-v2-3b",
                          tokenizer="databricks/dolly-v2-3b",
                          framework="pt",  # Explicitly specify the framework
                          torch_dtype=torch.bfloat16,
                          trust_remote_code=True,
                          device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

instruct_pipeline.py:   0%|          | 0.00/9.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/databricks/dolly-v2-3b:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Define a custom _forward method to fix the TypeError
def _forward(self, model_inputs, **generate_kwargs):
    input_ids = model_inputs.get("input_ids")
    attention_mask = model_inputs.get("attention_mask", None)

    generated_sequence = self.model.generate(
        input_ids=input_ids.to(self.model.device),
        attention_mask=attention_mask.to(self.model.device) if attention_mask is not None else None,
        **generate_kwargs,
    )

    return generated_sequence

# Bind the custom _forward method to the pipeline object
dolly_pipeline.forward = _forward.__get__(dolly_pipeline)

# Define a custom postprocess method to handle model outputs correctly
def postprocess(self, model_outputs, response_key_token_id, end_key_token_id, return_full_text: bool = False):
    generated_sequence = model_outputs[0]  # Access the first item in the model_outputs list
    instruction_text = None  # Placeholder for instruction_text
    # Additional processing if needed
    return generated_sequence, instruction_text


# Bind the custom postprocess method to the pipeline object
dolly_pipeline.postprocess = postprocess.__get__(dolly_pipeline)



In [ ]:
# define helper function
def get_completion_dolly(input):
    system = """
    You are an expert Physicist.
    You are good at explaining Physics concepts in simple words.
    Help as much as you can.
    """
    prompt = f"#### System: {system}\n#### User: \n{input}\n\n#### Response from Dolly-v2-3b:"
    print(prompt)
    dolly_response = dolly_pipeline(prompt,
                                    max_new_tokens=500,
                                    pad_token_id=dolly_pipeline.tokenizer.pad_token_id)  # Pass pad_token_id explicitly

    generated_sequence, instruction_text = dolly_response

    # Convert generated_sequence tensor to text using the tokenizer
    generated_text = dolly_pipeline.tokenizer.decode(generated_sequence.tolist(), skip_special_tokens=True)

    return generated_text

In [ ]:

# let's prompt
prompt = "Explain the difference between nuclear fission and fusion."
# prompt = "Why is the Sky blue?"

print(get_completion_dolly(prompt))

#### System: 
    You are an expert Physicist.
    You are good at explaining Physics concepts in simple words.
    Help as much as you can.
    
#### User: 
Explain the difference between nuclear fission and fusion.

#### Response from Dolly-v2-3b:
Below is an instruction that describes a task. Write a response that appropriately completes the request.


#### System: 
    You are an expert Physicist.
    You are good at explaining Physics concepts in simple words.
    Help as much as you can.
    
#### User: 
Explain the difference between nuclear fission and fusion.

#### Response from Dolly-v2-3b:


Nuclear fission creates more elemental particles than fusion.
Fusion creates one heavier element than its input material.




### Build AI Physics Tutor

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 11.5 MB/s

In [ ]:
import locale
# print(locale.getpreferredencoding())
locale.getpreferredencoding = lambda: "UTF-8"

# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working

In [ ]:
import gradio as gr
def get_completion(input):
    dolly_response = dolly_pipeline(prompt,
                                    max_new_tokens=500,
                                    pad_token_id=dolly_pipeline.tokenizer.pad_token_id)  # Pass pad_token_id explicitly

    generated_sequence, instruction_text = dolly_response

    # Convert generated_sequence tensor to text using the tokenizer
    generated_text = dolly_pipeline.tokenizer.decode(generated_sequence.tolist(), skip_special_tokens=True)

    return generated_text
# build an app front-end / ai tool with Gradio
iface = gr.Interface(fn=get_completion, inputs=[gr.Textbox(label="Insert Prompt Here",lines=6)],
                     outputs=[gr.Textbox(label="Your Answer Here",lines=3)],
                     title="My AI Physics Teacher",
                     examples=["Explain the difference between nuclear fusion and fission.",
                              "Why is the Sky blue?"]
                     )
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://350147ef407c41da9b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
from PIL import Image
import requests
from torchvision import models, transforms
import torch

# Load pre-trained ImageNet model
model = models.resnext50_32x4d(pretrained=True)
model.eval()

def classify_image(image):
    # Preprocess the image
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.fromarray(image.astype('uint8'), 'RGB')  # Convert numpy array to PIL image
    image = preprocess(image).unsqueeze(0)

    # Predict class probabilities
    with torch.no_grad():
        output = model(image)
        probabilities = torch.nn.functional.softmax(output[0], dim=0)

    # Get class labels
    imagenet_labels = requests.get("https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json").json()
    label_index = torch.argmax(probabilities).item()
    class_label = imagenet_labels[label_index]

    return class_label

iface = gr.Interface(fn=classify_image, inputs="image", outputs="text", title="Image Classifier")
iface.launch()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://425a5906ae8428a2fb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# TExt Sentiment Analysis
import gradio as gr
from transformers import pipeline

# Load sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

def analyze_sentiment(text):
    # Analyze sentiment
    result = sentiment_pipeline(text)
    return result[0]['label']

iface = gr.Interface(fn=analyze_sentiment, inputs="text", outputs="text", title="Text Sentiment Analysis")
iface.launch(share=True)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://8fc47e5ba9eac3b597.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# Language Translation
import gradio as gr
from transformers import MarianMTModel, MarianTokenizer

# Load MarianMT translation model
model_name = "Helsinki-NLP/opus-mt-en-fr"
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)

def translate_text(text):
    # Translate text from English to French
    inputs = tokenizer([text], return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

iface = gr.Interface(fn=translate_text, inputs="text", outputs="text", title="Language Translation-English to French")
iface.launch(share=True)


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e63c0bbf2d26c5beda.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
!pip uninstall -y whisper
!pip install openai-whisper
!pip install gradio soundfile scipy numpy

Found existing installation: whisper 1.1.10
Uninstalling whisper-1.1.10:
  Successfully uninstalled whisper-1.1.10
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.6 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801358 sha256=fb63a8cb4c326ec83bf77c9a9fa412cebdc66ce268a6d35c98629750c82f77d8
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper


In [ ]:
import whisper
import gradio as gr
import numpy as np
import tempfile
import soundfile as sf

# Load the Whisper model correctly
model = whisper.load_model("base")

# Function to perform speech recognition
def recognize_speech(audio):
    # Save the audio input to a temporary file
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
        sf.write(tmp.name, audio[1], audio[0])
        result = model.transcribe(tmp.name)
    return result["text"]

# Gradio interface
iface = gr.Interface(
    fn=recognize_speech,
    inputs=gr.Audio(type="numpy", label="Speak"),
    outputs=gr.Textbox(label="Transcription"),
    title="Speech Recognition",
    description="Speak into the microphone and get the transcription using OpenAI's Whisper model.",
)

iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f52016a638fb033b99.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
import os
import tensorflow as tf
import numpy as np
import PIL.Image
import tensorflow_hub as hub

# Function to load image from numpy array
def load_img(img):
    max_dim = 512
    img = tf.convert_to_tensor(img)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

# Function to convert tensor to image
def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor) > 3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

# Load the model from TensorFlow Hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

# Function to perform style transfer
def transfer_style(content_image, style_image):
    content_image = load_img(content_image)
    style_image = load_img(style_image)

    # Stylize content image with style image
    stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]

    return tensor_to_image(stylized_image)

# Create Gradio Interface
iface = gr.Interface(fn=transfer_style, inputs=["image", "image"], outputs="image", title="Image Style Transfer")
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1d404978b651c763a6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
